In [ ]:
###############################################################################
import aliases

In [ ]:
from everest.ptolemaic.ptolemaic import Ptolemaic
from everest.ptolemaic.sprite import Tuuple, Mapp
from everest.ptolemaic.resourcers import Resourcer, PtolContent

In [ ]:
import string
mytup = Tuuple(string.ascii_lowercase)
display(mytup)
mymap = Mapp(zip(mytup, range(len(mytup))))
display(mymap)

In [ ]:
class MyPtol(Ptolemaic):

    @classmethod
    def get_classproxy(cls, /):
        return 'Foo!'

In [ ]:
PtolContent(MyPtol)

In [ ]:
Resourcer(MyPtol)

In [ ]:
import pickle

In [ ]:
myptol = Ptolemaic()

In [ ]:
pickle.loads(pickle.dumps(myptol))

In [ ]:
pickle.loads(pickle.dumps(mymap))